In [37]:
import numpy as np
import pickle
import math
import matplotlib.pyplot as plt
import pandas as pd
import time
import pprint
import seaborn as sns
import CMR_IA as cmr
import scipy as sp
import pingouin as pg
import sys

sns.set_context("paper")
np.set_printoptions(suppress=True)
# pd.set_option('display.max_columns', None)

### Study Params

In [38]:
# # manually adjusted params
# # study params should be the same across three groups
# params.update(
#     beta_enc = 0.3,
#     gamma_fc = 0.2,
#     gamma_cf = 0.2,
#     s_fc = 0.2,
#     s_cf = 0.2,
#     phi_s = 1,
#     phi_d = 0.6,
#     d_ass = 1,
#     learn_while_retrieving = False,
#     var_enc = 1,
#     bad_enc_ratio = 1,
# )
# params.update(
#     beta_cue = 0.4,
#     beta_rec = 0.1,  # beta for retrieved item
#     beta_rec_post = 1,
#     beta_distract = 0.1,
#     # beta_rec_post = 0.1,
#     c_thresh_itm = 0.01,
#     c_thresh_ass = 0.01,
#     c_thresh = 0.01,
#     kappa = 0.01,
#     lamb = 0.002,
#     eta = 0.002,
#     omega = 3,
#     alpha = 1
# )
# params

In [39]:
sys.path.append("../../Modeling/CMR_IA/fitting")
from optimization_utils import make_boundary

# read pso results
_, _, what_to_fit = make_boundary(sim_name="S1")
optim_params = np.loadtxt("../../Modeling/CMR_IA/fitting/outparams_backup/S1_240725_200-200.txt")

# get params
# it seems that the size of all potential words matters, the more, the worse CR. RISK IT!
params = cmr.make_default_params()
for pname, pvalue in zip(what_to_fit, optim_params):
    params[pname] = pvalue
params.update(learn_while_retrieving=True, rec_time_limit=10000, use_new_context=True, d_ass=0)
params

{'beta_enc': 0.531592,
 'beta_rec': 0.000883,
 'beta_cue': 0.880728,
 'beta_rec_post': 0.912767,
 'beta_distract': 0.802725,
 'phi_s': 2.505553,
 'phi_d': 3.220387,
 's_cf': 0.026084,
 's_fc': 0.14797,
 'kappa': 0.298557,
 'eta': 0.001019,
 'omega': 9.243667,
 'alpha': 0.971336,
 'c_thresh': 0.222951,
 'c_thresh_itm': 0.279405,
 'c_thresh_ass': 0.481327,
 'd_ass': 0,
 'lamb': 0.059168,
 'rec_time_limit': 10000,
 'dt': 10,
 'nitems_in_accumulator': 50,
 'max_recalls': 50,
 'learn_while_retrieving': True,
 'a': 2800,
 'b': 20,
 'm': 0,
 'n': 1,
 'c1': 0,
 'thresh_sigma': 0,
 'No_recall': None,
 'var_enc': 1,
 'bad_enc_ratio': 1,
 'gamma_fc': 0.322544,
 'gamma_cf': 0.813101,
 'use_new_context': True}

### Simulation Function

In [40]:
def simu_success(tag, params):
    # which task
    path = "simuS1_data/simuS1_design.pkl"
    if tag == "Item-CR":
        # path = "../Data/simuS1_group3_design.pkl"
        nitems = 48 * 4  # 136
        test1_num = 80
        i = 1
    elif tag == "Pair-CR":
        # path = "../Data/simuS1_group2_design.pkl"
        nitems = 48 * 4  # 176
        test1_num = 80
        i = 2
    elif tag == "Asso-CR":
        # path = "../Data/simuS1_group1_design.pkl"
        nitems = 48 * 4  # 96
        test1_num = 40
        i = 3

    # load stimuli
    with open(path, "rb") as inp:
        df_study = pickle.load(inp)
        df_test = pickle.load(inp)
    df_study = df_study.query(f"group == {i}")
    df_test = df_test.query(f"group == {i}")

    # load semantic matrix
    s_mat = np.load("../../Data/wordpools/ltp_FR_similarity_matrix.npy")

    # update n_item in params to fit the study
    params.update(nitems_in_accumulator=nitems)
    # print(params)

    # run CMR
    df_simu, f_in_acc, f_in_dif = cmr.run_success_multi_sess(params, df_study, df_test, s_mat)
    df_simu["test"] = df_test["test"]
    df_simu = df_simu.merge(df_test, on=["session", "list", "test", "test_itemno1", "test_itemno2"])

    # get f_in
    sessions = np.unique(df_simu.session)
    tmp_corr_fin = []
    tmp_omax_fin = []

    for sess in sessions:
        df_tmp = df_study.loc[df_study.session == sess]
        tmp1 = df_tmp.study_itemno1.to_numpy()
        tmp2 = df_tmp.study_itemno2.to_numpy()
        df_tmp2 = df_test.loc[df_test.session == sess]
        tmp3 = df_tmp2.test_itemno1[df_tmp2.test_itemno1 >= 0].to_numpy()
        tmp4 = df_tmp2.test_itemno2[df_tmp2.test_itemno2 >= 0].to_numpy()
        tmp = np.concatenate((tmp1, tmp2, tmp3, tmp4))
        tmp = np.unique(tmp)  # sort
        nlists = len(np.unique(df_simu.list))

        for lst in range(nlists):
            tmp_corr = df_simu.query(f"session == {sess} and list == {lst}")["correct_ans"][test1_num:]
            corrid = np.searchsorted(tmp, tmp_corr)
            corr_fin = [f_in_dif[sess][lst * int(test1_num / 2) + i][id] for i, id in enumerate(corrid)]
            omax_fin = [np.max(np.delete(f_in_dif[sess][lst * int(test1_num / 2) + i], id)) for i, id in enumerate(corrid)]

            tmp_corr_fin = tmp_corr_fin + [-1] * test1_num + corr_fin
            tmp_omax_fin = tmp_omax_fin + [-1] * test1_num + omax_fin

    df_simu["corr_fin"] = tmp_corr_fin
    df_simu["omax_fin"] = tmp_omax_fin

    # optimal threshold for test1 (only when manually!!)
    # csim_two = df_simu.query("test==1").groupby("correct_ans").csim.mean()
    # opt_thresh = np.mean(csim_two)
    # df_simu['s_resp'] = df_simu.apply(lambda x: (1 if x['csim'] > opt_thresh else 0) if x['test'] == 1 else x['s_resp'], axis=1)

    return df_simu

### Analysis Function

In [41]:
# anal_perform used by pso
sys.path.append("../../Modeling/CMR_IA/fitting")
from object_funcs import anal_perform_S1 as anal_perform

In [42]:
# def anal_perform(df_simu):
#     # Get correctness
#     df_simu["correct"] = df_simu.s_resp == df_simu.correct_ans

#     # Recognition performance
#     df_recog = df_simu.query("test==1")
#     recog_resp = df_recog["s_resp"].to_numpy()
#     is_old = df_recog["correct_ans"].to_numpy()
#     is_new = 1 - is_old
#     old_num = np.sum(is_old)
#     new_num = np.sum(is_new)
#     hr = np.sum(recog_resp * is_old) / old_num
#     far = np.sum(recog_resp * is_new) / new_num

#     # Cued recall performance
#     df_cr = df_simu.query("test==2")
#     cr_resp = df_cr["s_resp"].to_numpy()
#     cr_truth = df_cr["correct_ans"].to_numpy()
#     p_rc = np.mean(cr_resp == cr_truth)

#     # successive test performance and calculate Q
#     df_simu_study = df_simu.query("pair_idx >= 0")
#     df_pair = pd.pivot_table(df_simu_study, index="pair_idx", columns="test", values="correct")
#     test1_resp = df_pair[1].to_numpy(dtype=int)
#     test2_resp = df_pair[2].to_numpy(dtype=int)
#     A = np.sum((test1_resp == 1) & (test2_resp == 1)) + 0.5
#     B = np.sum((test1_resp == 0) & (test2_resp == 1)) + 0.5
#     C = np.sum((test1_resp == 1) & (test2_resp == 0)) + 0.5
#     D = np.sum((test1_resp == 0) & (test2_resp == 0)) + 0.5
#     q = (A * D - B * C) / (A * D + B * C)

#     return p_rc, hr, far, q

### Item - CR (Group1)

In [43]:
tag = "Item-CR"
df_simu_g1 = simu_success(tag, params)
df_simu_g1

CMR2 Time: 96.34247088432312


,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,correct_ans,order,pair_idx,subject,group,test_item2,corr_fin,omax_fin
0,0,0,609,-1,0.0,2397.432129,0.271644,1,FROST,1,0,14,0,1,NaN,-1.000000,-1.000000
1,0,0,101,-1,0.0,365.569458,0.177609,1,BATTERY,0,-1,-1,0,1,NaN,-1.000000,-1.000000
2,0,0,927,-1,0.0,2328.979980,0.270196,1,MOVIE,1,1,13,0,1,NaN,-1.000000,-1.000000
3,0,0,1303,-1,1.0,663.443237,0.351402,1,SIDEWALK,1,0,31,0,1,NaN,-1.000000,-1.000000
4,0,0,945,-1,1.0,686.217773,0.349714,1,NEST,1,0,2,0,1,NaN,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47995,99,3,509,-1,-1.0,-1.000000,-1.000000,2,DUSTPAN,128,1,149,99,1,NaN,-0.287727,0.374345
47996,99,3,2,-1,-1.0,-1.000000,-1.000000,2,ACID,970,1,163,99,1,NaN,0.163888,0.222276
47997,99,3,1463,-1,-1.0,-1.000000,-1.000000,2,TICK,1255,0,171,99,1,NaN,0.165477,0.208573
47998,99,3,1615,-1,-1.0,-1.000000,-1.000000,2,WINNER,667,1,157,99,1,NaN,0.080424,-0.043659


In [44]:
subjects = np.unique(df_simu_g1.subject)
g1_stats = []
for subj in subjects:
    df_subj = df_simu_g1.query(f"subject == {subj}").copy()
    g1_stats.append(list(anal_perform(df_subj)))
g1_stats = np.array(g1_stats)

In [45]:
print(g1_stats)
print("mean:")
print(np.mean(g1_stats, axis=0))
print("se:")
print(sp.stats.sem(g1_stats, axis=0))

[[0.1375     0.65       0.2625     0.68197215]
 [0.1375     0.675      0.15625    0.64783128]
 [0.1375     0.65       0.11875    0.55418994]
 [0.1        0.725      0.15       0.41260745]
 [0.125      0.6375     0.13125    0.52892562]
 [0.14375    0.66875    0.1875     0.54342154]
 [0.125      0.59375    0.20625    0.59846547]
 [0.11875    0.65625    0.1875     0.61771058]
 [0.13125    0.59375    0.13125    0.61996497]
 [0.1375     0.69375    0.1875     0.47379114]
 [0.13125    0.725      0.30625    0.72069465]
 [0.14375    0.66875    0.1875     0.41584158]
 [0.11875    0.64375    0.14375    0.35112016]
 [0.1375     0.63125    0.18125    0.58468177]
 [0.1375     0.63125    0.05625    0.46417112]
 [0.125      0.7125     0.19375    0.38095238]
 [0.11875    0.66875    0.15625    0.59866962]
 [0.14375    0.6875     0.2375     0.509105  ]
 [0.11875    0.70625    0.1375     0.71449925]
 [0.125      0.675      0.1875     0.61008186]
 [0.11875    0.6375     0.16875    0.64449064]
 [0.14375    

In [46]:
# calculate d prime
g1_ds = sp.stats.norm.ppf(g1_stats[:, 1]) - sp.stats.norm.ppf(g1_stats[:, 2])
print(np.mean(g1_ds))
print(sp.stats.sem(g1_ds))

1.3880249382886516
0.01600575353455809


### Pair - CR (Group2)

In [47]:
tag = "Pair-CR"
df_simu_g2 = simu_success(tag, params)
df_simu_g2

CMR2 Time: 78.31911611557007


,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,correct_ans,order,pair_idx,subject,group,test_item2,corr_fin,omax_fin
0,0,0,192,1402,0.0,51.403214,0.479289,1,BULB,1,1,13,0,2,SUIT,-1.000000,-1.000000
1,0,0,810,1481,0.0,2309.799805,0.289028,1,LEMON,0,-1,-1,0,2,TOOTH,-1.000000,-1.000000
2,0,0,384,686,1.0,7.588796,0.574940,1,COTTAGE,1,1,8,0,2,HANDKERCHIEF,-1.000000,-1.000000
3,0,0,1317,1137,0.0,65.662468,0.091763,1,SKILLET,0,-1,-1,0,2,PUBLISHER,-1.000000,-1.000000
4,0,0,506,977,0.0,1188.863525,0.236574,1,DUNE,0,-1,-1,0,2,ONION,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35995,99,2,120,-1,-1.0,-1.000000,-1.000000,2,BELT,917,0,97,99,2,-1,0.349655,0.285083
35996,99,2,328,-1,-1.0,-1.000000,-1.000000,2,CLOUD,869,0,113,99,2,-1,-0.314285,0.097904
35997,99,2,456,-1,1582.0,4070.000000,0.726337,2,DIAGRAM,1582,0,110,99,2,-1,0.432481,0.250835
35998,99,2,655,-1,-1.0,-1.000000,-1.000000,2,GRASS,1520,0,114,99,2,-1,0.454858,0.464471


In [48]:
df_simu_g2.query("session == 0 and list == 0 and test == 1")

,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,correct_ans,order,pair_idx,subject,group,test_item2,corr_fin,omax_fin
0,0,0,192,1402,0.0,51.403214,0.479289,1,BULB,1,1,13,0,2,SUIT,-1.0,-1.0
1,0,0,810,1481,0.0,2309.799805,0.289028,1,LEMON,0,-1,-1,0,2,TOOTH,-1.0,-1.0
2,0,0,384,686,1.0,7.588796,0.574940,1,COTTAGE,1,1,8,0,2,HANDKERCHIEF,-1.0,-1.0
3,0,0,1317,1137,0.0,65.662468,0.091763,1,SKILLET,0,-1,-1,0,2,PUBLISHER,-1.0,-1.0
4,0,0,506,977,0.0,1188.863525,0.236574,1,DUNE,0,-1,-1,0,2,ONION,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0,0,1635,1509,1.0,38.999027,0.493097,1,ZEBRA,1,1,2,0,2,TROPHY,-1.0,-1.0
76,0,0,1490,1612,1.0,12.551702,0.549781,1,TOWER,1,1,34,0,2,WINDSHIELD,-1.0,-1.0
77,0,0,1153,1314,1.0,5.692108,0.589320,1,RACCOON,1,1,35,0,2,SKELETON,-1.0,-1.0
78,0,0,172,579,0.0,1815.986206,0.257755,1,BRICK,0,-1,-1,0,2,FLESH,-1.0,-1.0


In [49]:
subjects = np.unique(df_simu_g2.subject)
g2_stats = []
for subj in subjects:
    df_subj = df_simu_g2.query(f"subject == {subj}").copy()
    g2_stats.append(list(anal_perform(df_subj)))
g2_stats = np.array(g2_stats)

In [50]:
print(g2_stats)
print("mean:")
print(np.mean(g2_stats, axis=0))
print("se:")
print(sp.stats.sem(g2_stats, axis=0))

[[0.25       0.775      0.         0.63222908]
 [0.38333333 0.75833333 0.         0.74328922]
 [0.28333333 0.75       0.         0.61237785]
 [0.39166667 0.76666667 0.         0.73974703]
 [0.325      0.775      0.         0.63258786]
 [0.35       0.775      0.         0.67069486]
 [0.31666667 0.81666667 0.         0.51394204]
 [0.325      0.73333333 0.         0.79764244]
 [0.33333333 0.85833333 0.         0.74148297]
 [0.275      0.74166667 0.         0.72803347]
 [0.35833333 0.75833333 0.         0.71240527]
 [0.3        0.80833333 0.         0.6474181 ]
 [0.30833333 0.81666667 0.         0.78656801]
 [0.40833333 0.69166667 0.         0.84874676]
 [0.30833333 0.78333333 0.         0.70790378]
 [0.275      0.75       0.         0.83229814]
 [0.39166667 0.76666667 0.         0.73974703]
 [0.325      0.81666667 0.         0.66666667]
 [0.375      0.73333333 0.         0.77020826]
 [0.33333333 0.75       0.         0.6926549 ]
 [0.44166667 0.7        0.         0.86675862]
 [0.36666667 

In [51]:
# hack hr and far
old_num = 40
new_num = 40
g2_stats_hack = np.array(g2_stats)
hacked_hr = (g2_stats_hack[:, 1] * old_num + 0.5) / (old_num + 1)
hacked_far = (g2_stats_hack[:, 2] * new_num + 0.5) / (new_num + 1)
g2_stats_hack[:, 1] = hacked_hr
g2_stats_hack[:, 2] = hacked_far
g2_stats_hack

array([[0.25      , 0.76829268, 0.01219512, 0.63222908],
       [0.38333333, 0.75203252, 0.01219512, 0.74328922],
       [0.28333333, 0.74390244, 0.01219512, 0.61237785],
       [0.39166667, 0.7601626 , 0.01219512, 0.73974703],
       [0.325     , 0.76829268, 0.01219512, 0.63258786],
       [0.35      , 0.76829268, 0.01219512, 0.67069486],
       [0.31666667, 0.80894309, 0.01219512, 0.51394204],
       [0.325     , 0.72764228, 0.01219512, 0.79764244],
       [0.33333333, 0.8495935 , 0.01219512, 0.74148297],
       [0.275     , 0.73577236, 0.01219512, 0.72803347],
       [0.35833333, 0.75203252, 0.01219512, 0.71240527],
       [0.3       , 0.80081301, 0.01219512, 0.6474181 ],
       [0.30833333, 0.80894309, 0.01219512, 0.78656801],
       [0.40833333, 0.68699187, 0.01219512, 0.84874676],
       [0.30833333, 0.77642276, 0.01219512, 0.70790378],
       [0.275     , 0.74390244, 0.01219512, 0.83229814],
       [0.39166667, 0.7601626 , 0.01219512, 0.73974703],
       [0.325     , 0.80894309,

In [52]:
g2_stats = np.array(g2_stats)
g2_stats = g2_stats_hack
g2_ds = sp.stats.norm.ppf(g2_stats[:, 1]) - sp.stats.norm.ppf(g2_stats[:, 2])
print(np.mean(g2_ds))
print(sp.stats.sem(g2_ds))

2.9573785045827172
0.011773968774517819


### Association - CR (Group3)

In [53]:
tag = "Asso-CR"
df_simu_g3 = simu_success(tag, params)
df_simu_g3

CMR2 Time: 86.78613924980164


,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,correct_ans,order,pair_idx,subject,group,test_item2,corr_fin,omax_fin
0,0,0,1213,926,1.0,46.566727,0.484229,1,ROUGE,0,1,-1,0,3,MOUTH,-1.000000,-1.000000
1,0,0,830,717,0.0,59.082642,0.472327,1,LIST,1,1,34,0,3,HORIZON,-1.000000,-1.000000
2,0,0,577,1183,0.0,426.675018,0.373473,1,FLEA,1,1,0,0,3,RELISH,-1.000000,-1.000000
3,0,0,588,439,1.0,29.456783,0.507128,1,FOLDER,1,0,25,0,3,DANDRUFF,-1.000000,-1.000000
4,0,0,687,1311,1.0,14.901322,0.541201,1,HANGER,1,1,22,0,3,SIRLOIN,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,99,4,1169,-1,-1.0,-1.000000,-1.000000,2,RAT,331,0,226,99,3,-1,-0.290426,0.281422
29996,99,4,1349,-1,-1.0,-1.000000,-1.000000,2,SPHINX,848,1,219,99,3,-1,0.403181,0.323645
29997,99,4,1307,-1,-1.0,-1.000000,-1.000000,2,SILK,72,0,194,99,3,-1,0.371545,0.318029
29998,99,4,984,-1,899.0,3920.000000,0.710461,2,ORGAN,899,1,196,99,3,-1,0.435634,0.220042


In [54]:
subjects = np.unique(df_simu_g3.subject)
g3_stats = []
for subj in subjects:
    df_subj = df_simu_g3.query(f"subject == {subj}").copy()
    # df_subj = df_simu_g3.query(f"subject == {subj} and list == 0").copy()
    g3_stats.append(list(anal_perform(df_subj)))
g3_stats = np.array(g3_stats)

In [55]:
print(g3_stats)
print("mean:")
print(np.mean(g3_stats, axis=0))
print("se:")
print(sp.stats.sem(g3_stats, axis=0))

[[0.51       0.84       0.03       0.77991321]
 [0.39       0.69       0.07       0.86315789]
 [0.38       0.76       0.04       0.7804878 ]
 [0.5        0.75       0.08       0.88622259]
 [0.51       0.72       0.1        0.95254833]
 [0.4        0.78       0.06       0.86956522]
 [0.33       0.75       0.01       0.84852002]
 [0.43       0.81       0.06       0.9556196 ]
 [0.51       0.76       0.06       0.82538293]
 [0.4        0.7        0.09       0.92206186]
 [0.39       0.85       0.03       0.77362986]
 [0.5        0.81       0.05       0.89891135]
 [0.4        0.65       0.02       0.8486917 ]
 [0.4        0.81       0.04       0.72204473]
 [0.51       0.78       0.03       0.79714964]
 [0.49       0.88       0.04       0.80355699]
 [0.42       0.75       0.02       0.90313524]
 [0.44       0.8        0.03       0.78186969]
 [0.45       0.85       0.04       0.5544989 ]
 [0.41       0.78       0.02       0.7826087 ]
 [0.36       0.8        0.08       0.82051282]
 [0.37       

In [56]:
g3_ds = sp.stats.norm.ppf(g3_stats[:, 1]) - sp.stats.norm.ppf(g3_stats[:, 2])
print(np.mean(g3_ds[np.isfinite(g3_ds)]))
print(sp.stats.sem(g3_ds[np.isfinite(g3_ds)]))

2.4762878678335674
0.027341790412632218


### Aggregate Three Groups

In [57]:
stats = []
stats.append(list(np.mean(np.array(g1_stats), axis=0)))
stats.append(list(np.mean(np.array(g2_stats), axis=0)))
stats.append(list(np.mean(np.array(g3_stats), axis=0)))
stats = np.array(stats)
stats

array([[0.1308125 , 0.6639375 , 0.1709375 , 0.58342107],
       [0.32858333, 0.75853659, 0.01219512, 0.72127259],
       [0.4153    , 0.7789    , 0.0483    , 0.78524772]])

In [58]:
ground_truth = np.array([[0.19, 0.67, 0.15, 0.57], [0.30, 0.80, 0.12, 0.71], [0.42, 0.72, 0.22, 0.81]])  # p_rc, hr, far, q
err = np.mean(np.power(stats - ground_truth, 2))
err

0.004335705683065867

### Performance Analysis

In [59]:
def array2df(group_stats, group_num):
    group_stats = np.array(group_stats)
    df = pd.DataFrame(group_stats, columns=["PR", "HR", "FAR", "Q"])
    df["d"] = sp.stats.norm.ppf(np.array(group_stats[:, 1])) - sp.stats.norm.ppf(np.array(group_stats[:, 2]))
    df["subject"] = df.index
    df["group"] = group_num
    return df


df_group1 = array2df(g1_stats, 1)
df_group2 = array2df(g2_stats, 2)
df_group3 = array2df(g3_stats, 3)
df_groups = pd.concat([df_group1, df_group2, df_group3])
df_groups

,PR,HR,FAR,Q,d,subject,group
0,0.1375,0.6500,0.26250,0.681972,1.020977,0,1
1,0.1375,0.6750,0.15625,0.647831,1.463752,1,1
2,0.1375,0.6500,0.11875,0.554190,1.566579,2,1
3,0.1000,0.7250,0.15000,0.412607,1.634194,3,1
4,0.1250,0.6375,0.13125,0.528926,1.472286,4,1
...,...,...,...,...,...,...,...
95,0.4200,0.7400,0.02000,0.971554,2.697094,95,3
96,0.4900,0.7700,0.04000,0.792666,2.489533,96,3
97,0.4200,0.7300,0.06000,0.915107,2.167587,97,3
98,0.3700,0.7700,0.02000,0.755607,2.792596,98,3


In [60]:
pg.anova(data=df_groups, dv="PR", between="group", detailed=True)

,Source,SS,DF,MS,F,p-unc,np2
0,group,4.252207,2,2.126104,1138.431922,5.399775e-140,0.884609
1,Within,0.554669,297,0.001868,NaN,NaN,NaN


In [61]:
pg.pairwise_tukey(data=df_groups, dv="PR", between="group")

,A,B,mean(A),mean(B),diff,se,T,p-tukey,hedges
0,1,2,0.130812,0.328583,-0.197771,0.006112,-32.360003,0.0,-6.284892
1,1,3,0.130812,0.415300,-0.284488,0.006112,-46.548909,0.0,-6.510591
2,2,3,0.328583,0.415300,-0.086717,0.006112,-14.188905,0.0,-1.654858


In [62]:
pg.anova(data=df_groups.loc[np.isfinite(df_groups.d)], dv="d", between="group", detailed=True)

,Source,SS,DF,MS,F,p-unc,np2
0,group,129.205360,2,64.602680,1730.158064,1.092417e-163,0.921445
1,Within,11.015058,295,0.037339,NaN,NaN,NaN


In [63]:
pg.anova(data=df_groups, dv="Q", between="group", detailed=True)

,Source,SS,DF,MS,F,p-unc,np2
0,group,2.127662,2,1.063831,80.483444,1.176176e-28,0.351481
1,Within,3.925749,297,0.013218,NaN,NaN,NaN


In [64]:
pg.pairwise_tukey(data=df_groups, dv="Q", between="group")

,A,B,mean(A),mean(B),diff,se,T,p-tukey,hedges
0,1,2,0.583421,0.721273,-0.137852,0.016259,-8.478393,0.000000,-1.235628
1,1,3,0.583421,0.785248,-0.201827,0.016259,-12.413107,0.000000,-1.579190
2,2,3,0.721273,0.785248,-0.063975,0.016259,-3.934714,0.000305,-0.605152


### Symmetry Analysis

In [65]:
def test_sym(df_simu, testnum):
    df = df_simu.query(f"test == {testnum} and order >= 0").copy()
    df["correct"] = df["s_resp"] == df["correct_ans"]
    df_order = df.groupby(["order", "subject"]).correct.mean().to_frame(name="p_correct").reset_index()
    return df_order

In [66]:
df_simu_all = pd.concat([df_simu_g1, df_simu_g2, df_simu_g3])
df_order_recog = test_sym(df_simu_all, 1)
print(df_order_recog.groupby("order").p_correct.mean())
pg.pairwise_tests(df_order_recog, dv="p_correct", subject="subject", within="order")

order
0    0.774292
1    0.771917
Name: p_correct, dtype: float64


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,order,0,1,True,True,0.634144,99.0,two-sided,0.527449,0.135,0.086054


In [67]:
df_order_g1_recog = test_sym(df_simu_g1, 1)
print(df_order_g1_recog.groupby("order").p_correct.mean())
pg.pairwise_tests(df_order_g1_recog, dv="p_correct", subject="subject", within="order")

order
0    0.662250
1    0.665625
Name: p_correct, dtype: float64


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,order,0,1,True,True,-0.477772,99.0,two-sided,0.633866,0.124,-0.060538


In [68]:
df_order_g1_cr = test_sym(df_simu_g1, 2)
print(df_order_g1_cr.groupby("order").p_correct.mean())
pg.pairwise_tests(df_order_g1_cr, dv="p_correct", subject="subject", within="order")

order
0    0.131625
1    0.130000
Name: p_correct, dtype: float64


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,order,0,1,True,True,0.343338,99.0,two-sided,0.732072,0.117,0.060646


In [69]:
df_order_g2_recog = test_sym(df_simu_g2, 1)
print(df_order_g2_recog.groupby("order").p_correct.mean())
pg.pairwise_tests(df_order_g2_recog, dv="p_correct", subject="subject", within="order")

order
0    0.770167
1    0.759833
Name: p_correct, dtype: float64


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,order,0,1,True,True,1.503471,99.0,two-sided,0.135901,0.329,0.20424


In [70]:
df_order_g2_cr = test_sym(df_simu_g2, 2)
print(df_order_g2_cr.groupby("order").p_correct.mean())
pg.pairwise_tests(df_order_g2_cr, dv="p_correct", subject="subject", within="order")

order
0    0.332833
1    0.324333
Name: p_correct, dtype: float64


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,order,0,1,True,True,1.079181,99.0,two-sided,0.283129,0.195,0.145999


In [71]:
df_order_g3_recog = test_sym(df_simu_g3, 1)
print(df_order_g3_recog.groupby("order").p_correct.mean())
pg.pairwise_tests(df_order_g3_recog, dv="p_correct", subject="subject", within="order")

order
0    0.8664
1    0.8642
Name: p_correct, dtype: float64


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,order,0,1,True,True,0.483326,99.0,two-sided,0.629931,0.124,0.06606


In [72]:
df_order_g3_cr = test_sym(df_simu_g3, 2)
print(df_order_g3_cr.groupby("order").p_correct.mean())
pg.pairwise_tests(df_order_g3_cr, dv="p_correct", subject="subject", within="order")

order
0    0.414436
1    0.415864
Name: p_correct, dtype: float64


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,order,0,1,True,True,-0.128781,99.0,two-sided,0.897792,0.112,-0.017405
